# 02. langchain的快速入门
https://python.langchain.com/docs/get_started/quickstart

In [ ]:
from langchain.llms import OpenAI

# llm = OpenAI(openai_api_key="...") # 如果你有openai的api key，可以在这里填入
llm = OpenAI() # 这种写法是通过系统环境变量获取openai的api key

#### 环境变量设置
第一步，按下“win+r”键，打开运行框，

第二步，输入命令“control system”,

第三步，在控制面板主页，打开“高级系统设置”，

第四步，在高级系统设置界面，选择“环境变量”选项，

第五步，根据需求进行设置。

In [1]:
#设置代理
import os

os.environ['http_proxy'] = 'http://127.0.0.1:10809'
os.environ['https_proxy'] = 'http://127.0.0.1:10809'


## 2.1. 构建应用程序
现在我们可以开始构建语言模型应用程序了。LangChain提供了许多可用于构建语言模型应用程序的模块。模块可以在简单的应用程序中作为独立模块使用，并且可以组合用于更复杂的用例。在本节中，我们将介绍如何使用LangChain的模块来构建一个简单的应用程序。

LangChain应用程序的核心构建块是LLMChain。这结合了三件事：

    1.LLM：语言模型是这里的核心推理引擎。为了使用 LangChain，您需要了解不同类型的语言模型以及如何使用它们。

    2.提示模板：这为语言模型提供说明。这控制语言模型输出的内容，因此了解如何构造提示和不同的提示策略至关重要。

    3.输出解析器：这些将来自LLM的原始响应转换为更可行的格式，从而可以轻松使用下游的输出。

我们将单独介绍这三个组件，然后介绍组合所有这些组件的LLMChain。

了解这些概念将为您能够使用和自定义 LangChain 应用程序做好准备。

大多数LangChain应用程序都允许您配置LLM和/或使用的提示，因此知道如何利用这一点将是一个很大的推动因素。

### 2.1.1. LLMs
有两种类型的语言模型，在LangChain中称为：
```
llms：这是一个语言模型，它将字符串作为输入并返回一个字符串
chat_models：这是一种语言模型，它将消息列表作为输入并返回消息
```

LLM 的输入/输出简单易懂 - 一个字符串。

但是chat_models呢？

那里的输入是ChatMessage的列表 ，输出是单个ChatMessage .

一个ChatMessage有两个必需的组件：
```
content ：这是消息的内容。
role ：这是来自的 ChatMessage 实体的角色。
```
LangChain 提供了几个对象来轻松区分不同的角色：
```
HumanMessage ： ChatMessage来自人类/用户。

AIMessage ： ChatMessage来自AI/助手。

SystemMessage ：来自系统的ChatMessage 。

FunctionMessage ：来自函数调用的ChatMessage 。
```

LangChain 为两者公开了一个标准接口，但了解这种差异以便为给定语言模型构造提示很有用。LangChain 公开的标准接口有两种方法：

predict ：接受字符串，返回字符串

predict_messages ：接收消息列表，返回消息。

In [4]:
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI

llm = OpenAI()
chat_model = ChatOpenAI()

In [3]:
chat_model.predict("hi!")

'Hello! How can I assist you today?'

In [5]:
llm.predict("hi!")

'\\\\hi!");\n    var0.setSystemId("hi!");\n    java.lang.String var4 = var0.getSystemId();\n    int var5 = var0.getColumnNumber();\n    java.lang.String var7 = var0.lookupNamespaceURI("hi!");\n    var0.addAttribute("aa", "nu.staldal.lagoon.core.LagoonException: nu.staldal.lagoon.core.LagoonException: 4444444444", "nu.staldal.lagoon.core.LagoonException: nu.staldal.lagoon.util.TemplateException: ", "nu.staldal.lagoon.core.LagoonException~ nu.staldal.lagoon.util.TemplateException");\n    var0.setPublicId("nu.staldal.lagoon.core.LagoonException: nu_46_staldal_46_lagoon_46_core_46_LagoonException_58__32_4444444444");\n    // The following exception was thrown during execution.\n    // This behavior will recorded'

In [6]:
text = "What would be a good company name for a company that makes colorful socks?"

llm.predict(text)
# >> Feetful of Fun

'\n\nKaleidoscope Kicks.'

In [7]:
chat_model.predict(text)
# >> Socks O'Color

'1. ChromaSocks\n2. RainbowStride\n3. SockTone\n4. VibrantSole\n5. ColorPopSocks\n6. SpectrumSockCo\n7. ChromaticFeet\n8. KaleidoKicks\n9. ColorBurstSocks\n10. PrismaticSoles'

In [8]:
from langchain.schema import HumanMessage

text = "What would be a good company name for a company that makes colorful socks?"
messages = [HumanMessage(content=text)]

In [9]:
chat_model.predict_messages(messages)
# >> Socks O'Color

AIMessage(content='Rainbow Threads', additional_kwargs={}, example=False)

In [10]:
llm.predict_messages(messages)
# >> Feetful of Fun

AIMessage(content='\n\nSocktacular!', additional_kwargs={}, example=False)

对于这两种方法，还可以将参数作为关键字参数传入。

例如，您可以传入 temperature=0 以根据对象的配置调整使用的温度。在运行时传入的任何值都将始终覆盖对象配置的内容。

In [11]:
chat_model.predict_messages(messages,temperature=0)

AIMessage(content='VibrantSox', additional_kwargs={}, example=False)

In [12]:
chat_model.predict_messages(messages,temperature=0.5)

AIMessage(content='Socktastic', additional_kwargs={}, example=False)

In [13]:
chat_model.predict_messages(messages,temperature=1)

AIMessage(content='VividSocks', additional_kwargs={}, example=False)

### 2.1.2. 提示模板
大多数LLM应用程序不会将用户输入直接传递到LLM。
通常，他们会将用户输入添加到称为提示模板的较大文本中，该文本为手头的特定任务提供额外的上下文。

In [14]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate.from_template("What is a good name for a company that makes {product}?")
prompt.format(product="colorful socks")

'What is a good name for a company that makes colorful socks?'

提示模板还可用于生成消息列表。在这种情况下，提示不仅包含有关内容的信息，还包含每条消息（其角色，其在列表中的位置等）在这里，最常发生的是聊天提示模板是聊天消息模板的列表。
每个聊天消息模板都包含有关如何格式化该聊天消息的说明 - 它的角色，然后是它的内容。

In [16]:
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

template = "You are a helpful assistant that translates {input_language} to {output_language}."
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
human_template = "{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

chat_prompt = chat_prompt.format_messages(input_language="English", output_language="chinese", text="I love programming.")

chat_prompt

[SystemMessage(content='You are a helpful assistant that translates English to chinese.', additional_kwargs={}),
 HumanMessage(content='I love programming.', additional_kwargs={}, example=False)]

In [17]:
llm.predict_messages(chat_prompt)

AIMessage(content='\n\n系統：我愛編程。', additional_kwargs={}, example=False)

### 2.1.3. 输出解析器
输出解析器将LLM的原始输出转换为可以在下游使用的格式。输出分析器的主要类型很少，包括：


从LLM转换文本->结构化信息（例如JSON）

将聊天消息转换为字符串

将调用返回的额外信息转换为字符串，而不是消息（如 OpenAI 函数调用）。

In [18]:
from langchain.schema import BaseOutputParser

class CommaSeparatedListOutputParser(BaseOutputParser):
    """Parse the output of an LLM call to a comma-separated list."""


    def parse(self, text: str):
        """Parse the output of an LLM call."""
        return text.strip().split(", ")

CommaSeparatedListOutputParser().parse("hi, bye")


['hi', 'bye']

### 2.1.4. 大语言模型链
我们现在可以将所有这些组合成一个链。此链将获取输入变量，将这些变量传递给提示模板以创建提示，将提示传递给 LLM，然后通过（可选）输出分析器传递输出。这是捆绑模块化逻辑的便捷方法。让我们看看它的实际效果！

In [19]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.chains import LLMChain
from langchain.schema import BaseOutputParser

class CommaSeparatedListOutputParser(BaseOutputParser):
    """Parse the output of an LLM call to a comma-separated list."""


    def parse(self, text: str):
        """Parse the output of an LLM call."""
        return text.strip().split(", ")

template = """You are a helpful assistant who generates comma separated lists.
A user will pass in a category, and you should generate 5 objects in that category in a comma separated list.
ONLY return a comma separated list, and nothing more."""
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
human_template = "{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])
chain = LLMChain(
    llm=ChatOpenAI(),
    prompt=chat_prompt,
    output_parser=CommaSeparatedListOutputParser()
)
chain.run("colors")
# >> ['red', 'blue', 'green', 'yellow', 'orange']

['red', 'blue', 'green', 'yellow', 'orange']